In [1]:
from dgpy.dgsqlaio import DigiSql
from dgpy.dgsecrets import DigiSecret

In [2]:
supernova_config = {
    "flavor": 'mysql',
    "server": f'{DigiSecret()["DG_SHARED_ADDR"]}:{DigiSecret()["DG_SHARED_PORT"]}',
    "database": 'digikala',
    "username": DigiSecret()['DG_SHARED_USER'],
    "password": DigiSecret()['DG_SHARED_PASS']
}

upstream_root_config = {
    "flavor": 'mysql',
    "server": '172.16.159.34:10008',
    "database": 'mysql',
    "username": 'root',
    "password": 'x0J2_KQ3)SBu1vTY'
}

upstream_config = {
    "flavor": 'mysql',
    "server": '172.16.159.34:10008',
    "database": 'supernova',
    "username": 'root',
    "password": 'x0J2_KQ3)SBu1vTY'
}

downstream_config = {
    "flavor": 'sql-server',
    "server": '172.16.159.34:10009',
    "database": 'ods',
    "username": 'sa',
    "password": 'x0J2_KQ3)SBu1vTY'
}

### Create Database and Tables

We'll fetch the create statements for tables in shared, in order to be able to directly create them in the upstream database.
We'll do the same for the downstream database, and convert varchars to nvarchars.

In [3]:
with DigiSql(**upstream_root_config) as upstream:
    upstream.fetch('DROP DATABASE IF EXISTS supernova;')
    upstream.fetch('CREATE DATABASE supernova;')

In [4]:
tables = ['orders', 'order_items']

In [5]:
for table in tables:
    with DigiSql(**supernova_config) as supernova:
        create_statement = [line for line in supernova.fetch(f'SHOW CREATE TABLE {table}')][0][1]
        with DigiSql(**upstream_config) as upstream:
            upstream.fetch(create_statement)

with DigiSql(**upstream_config) as upstream:
    # upstream.fetch('CREATE TABLE offsets (table_name VARCHAR(50), offset_index int)')
    tables = upstream.fetch_all('SHOW TABLES;')

print(tables)

[('order_items',), ('orders',)]


In [9]:
with DigiSql(**upstream_config) as upstream:
    # upstream.fetch('CREATE TABLE offsets (table_name VARCHAR(50), offset_index int)')
    test = upstream.to_df('SELECT * FROM order_items;')
test

Empty DataFrame
Columns: []
Index: []

In [19]:
import pandas as pd

df = pd.DataFrame({
    'A': [1, 2, 3],
    'B': [4, 5, 7]
})
df.to_dict(orient = 'records')

[{'A': 1, 'B': 4}, {'A': 2, 'B': 5}, {'A': 3, 'B': 7}]